In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np
import glob
import os
from IPython.display import Image,display

import pickle
import time

Using TensorFlow backend.


In [2]:
from PIL import ImageDraw
def draw_bbox(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name']).convert('RGBA')
    
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    rect = PIL.Image.new('RGBA', img.size, (255,255,255,0))
    d = ImageDraw.Draw(img)
    d.rectangle(((lx,ly), (ux,uy)), outline="red")
    return PIL.Image.alpha_composite(img,rect)

def gen_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        display(draw_bbox(image_df.iloc[i]))
        display(draw_bbox(image_df.iloc[match]))

In [3]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

tops = bbox.iloc[cat[(cat.category_label>=1) & (cat.category_label<=20)].index]
bottoms = bbox.iloc[cat[(cat.category_label>=21) & (cat.category_label<=36)].index]
dresses = bbox.iloc[cat[(cat.category_label>36)].index]

In [4]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

In [5]:
def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

In [6]:
N = 10000
subset = np.random.randint(bbox.shape[0],size=N)
proc_img = []
for i in subset:
    proc_img.append(np.squeeze(preprocess_DF(bbox.iloc[i])))
proc_img = np.array(proc_img)

In [34]:
df = bbox.iloc[subset]

In [9]:
from sys import getsizeof
getsizeof(proc_img)

6021120144

In [10]:
import pickle
pickle.dump(bbox.iloc[subset],open('Data/feature_matrix/DF10000_bbox.p','wb'))

In [11]:
base_model = VGG16(weights='imagenet')
base_fc6 = Model(inputs=base_model.input,outputs=base_model.get_layer('fc1').output)
base_fc7 = Model(inputs=base_model.input,outputs=base_model.get_layer('fc2').output)

In [15]:
start = time.time()
fc6_base = base_fc6.predict(proc_img)
pickle.dump(fc6_base,open('Data/feature_matrix/FC6_base.p','wb'))
print('{.2f} s'.format([time.time()-start]))

AttributeError: 'list' object has no attribute '2f'

368 s


In [27]:
start = time.time()
fc7_base = base_fc7.predict(proc_img)
pickle.dump(fc7_base,open('Data/feature_matrix/FC7_base.p','wb'))
print('{:.0f} s'.format(time.time()-start))

188 s


In [29]:
from keras.models import load_model
trained_attr = load_model('DF-Category-trained-7-24.h5')
trained_fc6 = Model(inputs=trained_attr.input,outputs=trained_attr.get_layer('fc1').output)

In [30]:
start = time.time()
fc6_trained = trained_fc6.predict(proc_img)
pickle.dump(fc6_trained,open('Data/feature_matrix/FC6_trained.p','wb'))
print('{:.0f} s'.format(time.time()-start))

188 s


In [36]:
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')
df['category_label'] = cat.iloc[df.index]['category_label']

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
from PIL import ImageDraw
def draw_bbox(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name']).convert('RGBA')
    
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    rect = PIL.Image.new('RGBA', img.size, (255,255,255,0))
    d = ImageDraw.Draw(img)
    d.rectangle(((lx,ly), (ux,uy)), outline="red")
    return PIL.Image.alpha_composite(img,rect)

def gen_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        display(draw_bbox(image_df.iloc[i]))
        display(draw_bbox(image_df.iloc[match]))
        
def test_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    cat_match = []
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        cat_match.append(image_df.iloc[i]['category_label'] == image_df.iloc[match]['category_label'])
        
    return cat_match

def random_pairs(N,image_df):
    img_index = np.random.randint(image_df.shape[0],size=N)
    match_index = np.random.randint(image_df.shape[0],size=N)
    return (image_df.iloc[img_index]['category_label'].values
            == image_df.iloc[match_index]['category_label'].values)


In [43]:
def find_nearest(dist):
    j=1
    sort = np.argsort(dist)
        
    while dist[sort[j]]<=0:
        j += 1
        
    return sort[j]

def test_pairs_all_D(N,image_df,feature_matrix):
    img_index = np.random.randint(image_df.shape[0],size=N)
    cat_match = {'L2':[],'L1':[],'H':[]}
  
    bin_mat = (feature_matrix > 0)
    
    for i in img_index:
        img_feature = feature_matrix[i,:]
        L2_dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        L1_dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        img_feature = (img_feature>0)
        H_dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        L2_match = find_nearest(L2_dist)
        L1_match = find_nearest(L1_dist)
        H_match = find_nearest(H_dist)

        cat_match['L2'].append(image_df.iloc[i]['category_label'] == image_df.iloc[L2_match]['category_label'])
        cat_match['L1'].append(image_df.iloc[i]['category_label'] == image_df.iloc[L1_match]['category_label'])
        cat_match['H'].append(image_df.iloc[i]['category_label'] == image_df.iloc[H_match]['category_label'])
        
    return cat_match

In [37]:
r_score = random_pairs(1000,df)
print('P@1: {}'.format(np.mean(r_score)))

P@1: 0.116


In [39]:
start = time.time()
fc6_base_H_score = test_pairs(1000,df,fc6_base,'H')
print('P@1: {}'.format({np.mean(fc6_base_H_score)}))

P@1: {0.48799999999999999}


In [41]:
start = time.time()
fc7_base_H_score = test_pairs(1000,df,fc7_base,'H')
print('P@1: {}'.format(np.mean(fc7_base_H_score)))

P@1: 0.496


In [42]:
start = time.time()
fc6_trained_H_score = test_pairs(1000,df,fc6_trained,'H')
print('P@1: {}'.format(np.mean(fc6_trained_H_score)))

P@1: 0.505


In [51]:
fc6_trained_score = test_pairs_all_D(1000,df,fc6_trained)
print(np.mean(fc6_trained_score['H']))
print(np.mean(fc6_trained_score['L1']))
print(np.mean(fc6_trained_score['L2']))

0.56
0.49
0.51


In [50]:
print(np.mean(fc6_trained_score['H']))
print(np.mean(fc6_trained_score['L1']))
print(np.mean(fc6_trained_score['L2']))

0.45
0.45
0.47


In [48]:
np.mean(fc6_trained_H_score['H'])

0.56999999999999995